<a href="https://colab.research.google.com/github/ibnuazman/CarPlateRecognition/blob/main/ModelCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Car Plate Recognition System

## Step 1: Setting the environment

In [ ]:
# Mount your google drive here to use the resources

from google.colab import drive
drive.mount('/content/drive')

Using YOLOv5 to create the model to detect the car plate on the car. Install roboflow to import the training image dataset that have been anotated to detect the car plate in every picture.

In [ ]:
# Clone YOLOv5 and install roboflow
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="qTUwmR8g0ZpYGHEQBDgU")
project = rf.workspace("international-islamic-university-malaysia-izt9f").project("car-plate-njafw")
dataset = project.version(1).download("yolov5")

## Step 2: Train the dataset

Train the dataset using YOLOv5 with 300 epochs and yolov5s.pt model

In [ ]:
!python3 train.py --img 416 --batch 16 --epochs 300 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache

## Step 3: Model testing

Testing the dataset using the best model from previous training

In [ ]:
!python3 detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.5 --source {dataset.location}/test/images

In [ ]:
# Display inference on all test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

## Step 4: Convert pt model to onnx

Pytorch model (model.pt) will use CPU when running using Jetson Nano. The model need to be converted to onnx model (model.onnx) to run using Jetson Nano using GPU the make sure the program smooth.

In [ ]:
%cd /content

In [ ]:
!python3 /content/yolov5/export.py --weights runs/train/exp/weights/best.pt --batch-size 10 --imgsz 416 --include onnx --simplify

## Step 5: Setting the environment in the Jetson Nano and run the program main.py